In [49]:
## ddG check
import pandas as pd
import numpy as np
import pickle
from rdkit import Chem
from data_prepare import *

In [50]:
catalysts = pd.read_csv("../ASO/Catalysts/catalysts.csv")
imines = pd.read_csv("../ASO/Imines/imines.csv")
thiols = pd.read_csv("../ASO/Thiols/thiols.csv")

d = {}
for df in [catalysts, imines, thiols]:
    d.update(dict([(Chem.MolToInchi(Chem.MolFromSmiles(smi)), str(label)) for smi, label in zip(df["smiles"], df["label"]) if label != "ref"]))

In [51]:
ori_data = pd.read_excel("Denmark_data_original.xlsx", sheet_name="performance")
ori = {}
for c, i, t, ddG in zip(ori_data["Catalyst"], ori_data["Imine"], ori_data["Thiol"], ori_data["Output"]):
    cat = Chem.MolToInchi(Chem.MolFromSmiles(c))
    imi = Chem.MolToInchi(Chem.MolFromSmiles(i))
    thi = Chem.MolToInchi(Chem.MolFromSmiles(t))
    try:
        ori[f"{d[cat]}_{d[imi]}_{d[thi]}"] = ddG
    except Exception as e:
        ori[f"328_vi_{d[imi]}_{d[thi]}"] = ddG

In [52]:
ee = pd.read_excel("Reaction_data.xlsx", index_col=0)["% ee"]
ee.index = [x.replace("202_i", "202_vi") for x in ee.index]
R = 1.98718
T = 298.15
er = (ee + 100) / (100 - ee)
ddG = R * T * np.log(er) / 1000
my = ddG.to_dict()

In [53]:
err = np.array([abs(ori[key]-my[key]) for key in ori.keys()])
print(err.max())

0.0015405859378523346


In [54]:
df = []
for reaction, ddG in ori.items():
    df.append({"Reaction": reaction, "ddG": ddG})
df = pd.DataFrame(df)
df.to_csv("Reaction_data.csv", index=False)